In [1]:
import zfit
import tensorflow as tf
import numpy as np

In [2]:
obs = zfit.Space('x', limits=(-5, 5))

mu = zfit.Parameter('mu', 0, -1, 1)
sigma = zfit.Parameter('sigma', 1, 0.5, 1.5)
model = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma)

In [3]:
# do i need this range here?
nsig_toy = 1e4
nsig_range = 0.1
n_sig = zfit.Parameter('n_sig', nsig_toy, nsig_toy*(1-nsig_range), nsig_toy*(1+nsig_range))
model = model.create_extended(n_sig)

# vary only n_sig
sampler = model.create_sampler(fixed_params=[mu, sigma])
nll = zfit.loss.ExtendedUnbinnedNLL(model, sampler)

minimizer = zfit.minimize.Minuit(
    strategy=zfit.minimize.DefaultToyStrategy(),
    verbosity=0,
    tolerance=1e-3,
    use_minuit_grad=True)
params = nll.get_params()

In [4]:
fit_results = []
ntoys = 10

while len(fit_results) < ntoys:

    # What does really happen here?
    sampler.resample()

    # I think this is probably not what I want but I don't understand why
    # this is needed in the first place.
    # I want something like: vary n_sig randomly, keep mu and sigma fixed
    for param in params:
       param.randomize()

    result = minimizer.minimize(nll)

    if result.converged:
        result.hesse()
        fit_results.append(result)

# Pull plots
Now I want something like this:  
    pull =  (N_generated - N_fit ) / sigma_fit  
for each fit_result.